# MNIST
Rambaldelli Guglielmo

# Required preparation

In [ ]:
#install libray required
import os
print(os.system('sudo apt-get install python-numpy python-scipy python-sklearn python-pip python-matplotlib'))
print(os.system('pip3 install mlxtend tensorflow keras'))

In [ ]:
# create directory and download database
import os
os.chdir('/tmp')
os.system('mkdir AML')
%cd 'AML'
os.system('curl -o  train-images-idx3-ubyte.gz http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz')
os.system('curl -o train-labels-idx1-ubyte.gz http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz')
os.system('curl -o t10k-images-idx3-ubyte.gz http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz')
os.system('curl -o t10k-labels-idx1-ubyte.gz http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz')
os.system('gunzip train-images-idx3-ubyte.gz')
os.system('gunzip train-labels-idx1-ubyte.gz')
os.system('gunzip t10k-images-idx3-ubyte.gz')
os.system('gunzip t10k-labels-idx1-ubyte.gz')

In [ ]:
#Import required 
from mlxtend.data import loadlocal_mnist

from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression                         
from sklearn.tree import DecisionTreeClassifier                             
from sklearn.neighbors import KNeighborsClassifier                          
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis        
from sklearn.naive_bayes import GaussianNB                                  
from sklearn.svm import SVC         
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import OneClassSVM


from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from sklearn.metrics import classification_report  
from sklearn.metrics import confusion_matrix 

from matplotlib import pyplot

In [ ]:
#Load test and train set

X_train, Y_train = loadlocal_mnist(
        images_path='train-images-idx3-ubyte', 
        labels_path='train-labels-idx1-ubyte')

X_test, Y_test = loadlocal_mnist(
        images_path='t10k-images-idx3-ubyte', 
        labels_path='t10k-labels-idx1-ubyte')


# Model selection

First af all we tried to compare the various algorithm we studied on this specific problem to see wich can be suitablie.

In [ ]:
models = []
models.append(( 'LR'   , LogisticRegression()))
models.append(( 'LDA'  , LinearDiscriminantAnalysis()))
models.append(( 'KNN'  , KNeighborsClassifier()))
models.append(( 'CART' , DecisionTreeClassifier()))
models.append(( 'NB'   , GaussianNB()))
models.append(( 'SVM'  , SVC()))
models.append(( 'MLP'  , MLPClassifier()))
models.append(( 'RF'  , RandomForestClassifier()))


results = []
names = []
scoring = 'accuracy'
for name, model in models:
  model.fit(X_train, Y_train)
  cv_results =  model.score(X_test, Y_test) 
  results.append(cv_results)
  names.append(name)
  msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
  print(msg)
    

# Support Vector Machines


From the literature we saw that a succesfull aproac used was SVM so we experimanted with it.

In [ ]:
#Support Vector
models = []
models.append(( 'linear svc'   ,sklearn.svm.LinearSVC()))
models.append(( 'Nu'  jupyter notebook,sklearn.svm.NuSVC()))
models.append(( 'Svc'  ,sklearn.svm.SVC()))


results = []
names = []
scoring = 'accuracy'
for name, model in models:
    model.fit(X_train, Y_train)
    cv_results =  model.score(X_test, Y_test)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

In [ ]:
We saw that SVC had the best perforance so we tried a grid search.

In [ ]:
C=[0.1,0.5,1,2,5,10]
D=[3,4,5,6,7,8,9,10]
seed=[1234,5678,7890,3456,2345,6789]

with open("SVMcomp.txt", "w") as SVMcomp:
    for s in S:
        for c in C:
                for d in D:
                        model = sklearn.svm.SVC(C=c,degree=d,random_state=s  )  
                        # Evaluate using Cross Validation
                        num_folds = 5
                        seed = 1234
                        kfold = KFold(n_splits=num_folds, random_state=seed)
                        results = cross_val_score(model, X_train, Y_train, cv=kfold)
                        SVMcomp.write(x,s,results.mean()*100.0, results.std()*100.0)


# Other Methods

As the best one seems to be KNN, SVM, RF and MLP we keept experimenting on them.

In [ ]:
#Random Forest
s=[1,10,100]
for s in size:
    model = RandomForestClassifier(n_estimators=100, n_jobs=-1, )        
    model.fit(X_train, Y_train)             
    print(s,model.score(X_test,Y_test))

In [ ]:
#KNN
s=[1,10,100]
for s in size:
    model = KNeighborsClassifier(leaf_size=10,n_neighbors=10, weights='distance',n_jobs=-1 ) 
    model.fit(X_train, Y_train)             
    print(s,model.score(X_test,Y_test))

Unfortunatly with KNN, RF and SVM we could not reach the same accuracy that MLP could.

# Voting Classifier

We tried to complement them using a Voting System both with 'hard' and 'soft' voting but the imbalance in the methods used gave us worst result then single method as expected.

In [ ]:
#Voting CLassifier
LR=LogisticRegression()
LDA=LinearDiscriminantAnalysis()
KNN=KNeighborsClassifier()
CART=DecisionTreeClassifier()
NB=GaussianNB()
SVC=SVC()
MLP=MLPClassifier()
RF=RandomForestClassifier()

model = VotingClassifier(estimators=[
        ('LR', LR), ('LDA', LDA), ('KNN', KNN), ('CART', CART), ('NB', NB), ('SVC', SVC), ('MLP', MLP), ('RF', RF)], voting='hard')



In [ ]:
# Evaluate using Cross Validation
num_folds = 2
seed = 1234
kfold = KFold(n_splits=num_folds, random_state=seed)
results = cross_val_score(model, X_train, Y_train, cv=kfold)
print("Accuracy: %.3f%% (%.3f%%)" % (results.mean()*100.0, results.std()*100.0))


#test
model = model.fit(X_train, Y_train)
print(model.score('test',X_test,Y_test))

# Multi-layer Perceptron classifier 

After some testing we saw that MPL was the best performing method so decided to try optimize MLP Classifier.
(Warning: really long execution time)


In [ ]:
#MLP comparison


layers=[(500,300,100),(300,500,100),(100,300,500),(100,500,100),(100,500,300),(500,1000,500),(100,500,1000,100),(100,1000,500,100),(50,1000,100),(728,1000,100),(728,100,500,100),(728,100,1000,50),(728,364,1500,100),(728,10,100,10),(728,500,100)]

seed=['1234','2345','3456','4567','5678','6789','7890','8901','9012']

for x in layers:
        for s in seed:
            with open('MLPcomp.txt','w') as out:
                    g=int(s)
                    print(s,g)
                    model = MLPClassifier(hidden_layer_sizes=x, max_iter=1000,learning_rate_init=0.001,random_state=g)
                    # Evaluate using Cross Validation
                    num_folds = 2
                    seedk = 1234
                    kfold = KFold(n_splits=num_folds, random_state=seedk)
                    results = cross_val_score(model, X_train, Y_train, cv=kfold)
                    print(x, s, results.mean()*100.0, results.std()*100.0)
                    out.write(x+' '+s+' '+results.mean()*100.0+' '+results.std()*100.0)

In [ ]:
#select good (over 97 %) one and phrase them
print(os.popen('cat MLPcomp.txt | awk '$NF > 97 {print}' > clean.txt'))

In [ ]:
#best setup
layers=[(500,300,100),(300,500,100),(100,300,500),(100,500,100),(100,500,300),(500,1000,500),(100,500,1000,100),(100,1000,500,100),(50,1000,100),(728,1000,100),(728,100,500,100),(728,100,1000,50),(728,364,1500,100),(728,10,100,10),(728,500,100)]


(728, 364, 728, 364) 1234 97.00666666666666 0.12666666666666937
(728, 364, 728, 364) 3456 97.09 0.4500000000000004
(728, 364, 728, 364) 5678 97.22666666666666 0.0033333333333385173
(728, 50, 75, 100, 50) 1234 97.1 0.009999999999998899
(728, 50, 75, 100, 50) 3456 97.22500000000001 0.10833333333333806
(728, 50, 75, 100, 50) 4567 97.255 0.10166666666666657
(728, 50, 75, 100, 50) 7890 97.12833333333333 0.22166666666667
(728, 50, 75, 100, 50) 8901 97.12 0.06666666666666488
(728, 500, 300, 200, 100) 1234 97.11333333333334 0.1233333333333364
(728, 500, 300, 200, 100) 7890 97.37 0.13333333333332975
(728, 300, 500, 200, 100) 4567 97.01166666666667 0.07833333333333026
(728, 300, 500, 200, 100) 7890 97.30833333333334 0.24500000000000077
(728, 300, 500, 200, 100) 9012 97.12833333333333 0.11166666666667102
(728, 100, 200, 300, 100) 2345 97.315 0.22833333333333594
(728, 100, 200, 300, 100) 3456 97.30666666666667 0.02666666666666928
(728, 100, 200, 300, 100) 4567 97.06666666666666 0.1433333333333342
(728, 100, 200, 300, 100) 6789 97.125 0.3416666666666679
(728, 364, 728, 364) 1234 97.00666666666666 0.12666666666666937
(728, 364, 728, 364) 3456 97.09 0.4500000000000004
(728, 364, 728, 364) 5678 97.22666666666666 0.0033333333333385173

In [ ]:
#Setup voting classifiers using good performing layout
with open('clean.txt','r') as best:
    MLP=[]
    bestL= best.readlines()
    name=0
    for m in bestL:
        l=m.split(' ')
        seed=int(l[-3])
        layer=eval(' '.join(l[0:-3]))
        mlp=MLPClassifier(hidden_layer_sizes=layer, max_iter=1000,learning_rate_init=0.001,random_state=seed)
        MLP.append((str(name),mlp))
        name=name+1





In [ ]:
#Voting CLassifier with MLP (hard)
model = VotingClassifier(estimators=MLP,  voting='hard')

In [ ]:
#Voting CLassifier with MLP (soft)
model = VotingClassifier(estimators=MLP,  voting='soft')

In [ ]:
#test
model = model.fit(X_train, Y_train)
print(model.score('test',X_test,Y_test))

In [ ]:
#metrics
predicted = model.predict(X_test)
report = classification_report(Y_test, predicted) #report

matrix = confusion_matrix(Y_test, predicted) #confusion matrix

Using both hard and soft voting we managed to reach an accuracy of 98.8 %







# Convolutional NN


We saw from litterature that the best way to aproach this kind of problems are convolutional neural network,
given that sklearn dose not support them we decided to use keras and tensorflow as a backhand.

In [ ]:
import tensorflow
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from keras.datasets import mnist
from keras.utils import to_categorical
from keras import optimizers

#download mnist data and split into train and test sets
X_train, Y_train = loadlocal_mnist(
        images_path='train-images-idx3-ubyte', 
        labels_path='train-labels-idx1-ubyte')

X_test, Y_test = loadlocal_mnist(
        images_path='t10k-images-idx3-ubyte', 
        labels_path='t10k-labels-idx1-ubyte')

#reshape data to fit model
X_train = X_train.reshape(60000,28,28,1)
X_test = X_test.reshape(10000,28,28,1)
#one-hot encode target column
Y_train = to_categorical(Y_train)
Y_test = to_categorical(Y_test)
#create model
model = Sequential()
#add model layers
model.add(Conv2D(32, (3, 3), activation='sigmoid'))
model.add(Conv2D(32, (3, 3), activation='sigmoid'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.05))
model.add(Flatten())
model.add(Dense(100, activation='sigmoid'))
model.add(Dense(10, activation='softmax'))
#compile model using accuracy to measure model performance
opt = optimizers.SGD(lr=0.01, momentum=0.9)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

 
#train the model
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=10)

Using convolutional network we managed to reach an accuracy of 99.2%